## Importing Libraries

In [50]:
import os
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr
import glob
from pathlib import Path
from IPython.display import Markdown, display, update_display

In [37]:
load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")

OpenAI API Key exists and begins sk-proj-


In [38]:
MODEL = "gpt-4.1-nano"
openai = OpenAI()

## Reading the contents of files in dictionary 

In [86]:
knowledge = {}

filenames = glob.glob('../knowledge-base/employees/*')

for filename in filenames:
    name = Path(filename).stem.split(' ')[-1]
    with open(filename, 'r', encoding = 'utf-8') as f:
        knowledge[name.lower()] = f.read()

In [ ]:
knowledge['lancaster']

In [88]:
filenames = glob.glob('../knowledge-base/products/*')

for filename in filenames:
    name = Path(filename).stem.split(' ')[-1]
    with open(filename, 'r', encoding = 'utf-8') as f:
        knowledge[name.lower()] = f.read()

In [89]:
knowledge.keys()

dict_keys(['kim', 'patel', 'sharma', 'chen', 'spencer', 'foster', 'tran', 'blake', 'zhang', 'thompson', 'adams', 'liu', 'lancaster', 'park', 'greene', 'johnson', 'thomson', "o'brien", 'martinez', 'williams', 'rivera', 'trenton', 'anderson', 'harper', 'rodriguez', 'wilson', 'bishop', 'carter', 'brooks', 'walker', 'rellm', 'claimllm', 'bizllm', 'lifellm', 'healthllm', 'markellm', 'homellm', 'carllm'])

## Prompts

In [90]:
SYSTEM_PREFIX = """
You represent Insurellm, the Insurance Tech company.
You are an expert in answering questions about Insurellm; its employees and its products.
You are provided with additional context that might be relevant to the user's question.
Give brief, accurate answers. If you don't know the answer, say so.

Relevant context:
"""

In [102]:
def get_relevant_contexts(message):
    info = ''.join(ch for ch in message if ch.isalpha() or ch.isspace())
    words = info.lower().split()
    return [knowledge[word] for word in words if word in knowledge]

In [103]:
def additional_context(message):
    relevant_message = get_relevant_contexts(message)
    if not relevant_message:
        result = "There is no additional context relevant to the user's question."
    else:
        result = "The following additional context might be relevant in answering the user's question:\n\n"
        result += '\n\n'.join(relevant_message)
    return result

## RAG Setup:

In [115]:
def chat(message,history):
    messages = [{'role':'system', 'content':SYSTEM_PREFIX},
    {'role':'system','content':additional_context(message)}]

    for user_message, assistant_message , *rest in history:
        messages.append({'role':'user','content':user_message})
        messages.append({'role':'assistant','content':assistant_message})
    messages.append({'role':'user','content':message})

    response = openai.chat.completions.create(
        model = MODEL,
        messages = messages
    )

    result = response.choices[0].message.content
    return result

## Gradio UI

In [116]:
gr.ChatInterface(fn = chat).launch(inbrowser=True)

/opt/anaconda3/envs/llms/lib/python3.11/site-packages/gradio/chat_interface.py:345: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.
